In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
dircs=os.listdir('/kaggle/input/fishdataset/combine/FishDisease/')

In [ ]:
X_directory=[]
y_directory=[]
for directory in dircs:
    image_paths=os.listdir(f'/kaggle/input/fishdataset/combine/FishDisease/{directory}')
    for img_name in image_paths:
        X_directory.append(f'/kaggle/input/fishdataset/combine/FishDisease/{directory}/{img_name}')
        y_directory.append(directory)

In [ ]:
df={"fishimages":X_directory,"fishlabels":y_directory}

In [ ]:
df = pd.DataFrame(df)
df.style

In [ ]:
import pandas as pd
import numpy as np
df=pd.DataFrame(df)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
df = df.sample(frac=1).reset_index(drop=True)
onehot=OneHotEncoder()
lab=onehot.fit_transform(df['fishlabels'].values.reshape(-1,1)).toarray()
images=df['fishimages']
print('total labels of images',len(lab))
print('total images',len(images))

In [ ]:
import cv2
import matplotlib.pyplot as plt
imgs=[]
for img in images:
    if img.startswith("/kaggle/input/fishdataset/combine/FishDisease/white spot/"):
        other_imag = cv2.imread(img)

        other_frame = cv2.resize(other_imag,(224,224))
        kernel = np.array([[0, -1, 0],
               [-1, 5,-1],
               [0, -1, 0]])
        image_sharp = cv2.filter2D(src=other_frame, ddepth=-1, kernel=kernel)
        hsv = cv2.cvtColor(image_sharp, cv2.COLOR_BGR2HSV)

        # define range of white color in HSV
        # change it according to your need !
        lower_white = np.array([0,0,168], dtype=np.uint8)
        upper_white = np.array([172,111,255],dtype=np.uint8)

        # Threshold the HSV image to get only white colors
        mask = cv2.inRange(hsv, lower_white, upper_white)
        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(image_sharp,image_sharp, mask= mask)
        imgs.append(res)
         
    elif img.startswith("/kaggle/input/fishdataset/combine/FishDisease/red spot/"):
        imag = cv2.imread(img)

        frame = cv2.resize(imag,(224,224)) 

        kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
        image_sharp = cv2.filter2D(src=frame, ddepth=-1, kernel=kernel)
        hsv = cv2.cvtColor(image_sharp,cv2.COLOR_BGR2HSV)

        lower_red = np.array([0,50,50])
        upper_red = np.array([10,255,255])


        #upper red
        lower_red2 = np.array([170,50,50])
        upper_red2 = np.array([180,255,255])

        mask = cv2.inRange(hsv, lower_red, upper_red)
        res = cv2.bitwise_and(image_sharp,image_sharp, mask= mask)


        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        res2 = cv2.bitwise_and(image_sharp,image_sharp, mask= mask2)

        red=res+res2
        imgs.append(red)
    elif img.startswith("/kaggle/input/fishdataset/combine/FishDisease/black spot/"):
        other_imag = cv2.imread(img)

        other_frame = cv2.resize(other_imag,(224,224))
        kernel = np.array([[0, -1, 0],
               [-1, 5,-1],
               [0, -1, 0]])
        image_sharp = cv2.filter2D(src=other_frame, ddepth=-1, kernel=kernel)
        imagehsv = cv2.cvtColor(image_sharp, cv2.COLOR_BGR2HSV)


        lower_black = np.array([0,0,0])
        upper_black = np.array([179,255,127])

        imagemask = cv2.inRange(imagehsv, lower_black, upper_black)
        result = cv2.bitwise_not(image_sharp,image_sharp, mask=imagemask)

        imgs.append(result)
    elif img.startswith("/kaggle/input/fishdataset/combine/FishDisease/fresh fish/"):
        other_imag = cv2.imread(img)

        other_frame = cv2.resize(other_imag,(224,224))
        kernel = np.array([[0, -1, 0],
               [-1, 5,-1],
               [0, -1, 0]])
        image_sharp = cv2.filter2D(src=other_frame, ddepth=-1, kernel=kernel)

        imgs.append(image_sharp)

In [ ]:
X_preprocess=np.array(imgs)
y_preprocess=np.array(lab)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mimg
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)
from PIL import Image
from scipy import misc

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:

train_x, test_x, train_y, test_y = train_test_split(X_preprocess, y_preprocess, random_state = 42, 
                                                   test_size=0.30)

In [ ]:
X_train_norm=train_x/255.0
X_test_norm=test_x/255.0

In [ ]:
y_single_dim=np.argmax(train_y,axis=1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=128, max_value=256, step=32),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(224,224,3)),
    # adding second convolutional layer 
    
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
#     keras.layers.Dense(
#         units=640,
#         activation='relu'
#     ),
    # output layer    
    keras.layers.Dense(4, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
from kerastuner import RandomSearch
from keras.callbacks import EarlyStopping
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5)
# search best parameter
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
tuner.search(X_train_norm, train_y, epochs=30,batch_size=20, validation_data=(X_test_norm, test_y),callbacks=[es,mc])

In [ ]:
# import keras_tuner as kt
# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = keras.Sequential()
        
#         model.add(layers.Flatten(input_shape=(224,224,3)))
#         model.add(
#             layers.Dense(
#                 units=1024,
#                 activation="relu",
#             )
#         )
#         model.add(layers.Dense(4, activation="softmax"))
#         model.compile(
#             optimizer=keras.optimizers.Adam(0.001), loss="categorical_crossentropy", metrics=["accuracy"],
#         )
#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             epochs=hp.Choice("epochs", [30,50]),
#             batch_size=hp.Choice("batch_size", [32]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )

In [ ]:
# def build_model(hp):
    
#         model = keras.Sequential()
#         model.add(layers.Flatten(input_shape=(224,224,3)))
#         model.add(layers.Dense(units=hp.Int('units', min_value=512, max_value=1024, step=128), activation='relu'))
        
#         model.add(layers.Dense(4, activation='softmax'))
#         model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),
#             loss = 'categorical_crossentropy', metrics = ['accuracy'])
#         return model

In [ ]:
from kerastuner import RandomSearch
from keras.callbacks import EarlyStopping
# creating randomsearch object
# tuner = RandomSearch(build_model,
#                     objective='val_accuracy',
#                     max_trials = 5)
# search best parameter
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

tuner.search(X_train_norm, train_y,batch_size=20,epochs=30 validation_data=(X_test_norm, test_y),callbacks=[es])

In [ ]:
tuner.search_space_summary()

In [ ]:
# tuner.search(X_train_norm, train_y, epochs=30, validation_data=(X_test_norm, test_y))

In [ ]:
tuner.results_summary()

###  parameter tuning 70-30
Trial summary
Hyperparameters:
units: 1024
learning_rate: 0.001


## 8-32-8
conv_1_filter: 24
conv_1_kernel: 5
Score: 0.9060240983963013
## 32-128-16
conv_1_filter: 96
conv_1_kernel: 5
Score: 0.891566276550293
## 128-256-32
Hyperparameters:
conv_1_filter: 192
conv_1_kernel: 3
Score: 0.9228915572166443

## Epochs 30-50-16
epochs: 30
batch_size: 16
Score: 0.7108433842658997
## Epochs 30-50-32
epochs: 30
batch_size: 32
Score: 0.6963855624198914
    
## Epochs 30-50-20
epochs: 30
batch_size: 20
Score: 0.7855421900749207


